In [3]:
%pip install -U langchain-community langgraph langchain-openai tavily-python langgraph-checkpoint-sqlite

  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.4 MB/s eta 0:00:00a 0:00:01
Using cached langchain-0.3.23-py3-none-any.whl (1.0 MB)
Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.3.12
    Uninstalling langchain-openai-0.3.12:
      Successfully uninstalled langchain-openai-0.3.12
  Attempting uninstall: langchain
  

In [17]:
import os
import getpass

# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API Key: ")

# Set Tavily API key
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API Key: ")

Enter your LangSmith API Key:  ········
Enter your Tavily API Key:  ········


In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults
# Imports the built-in Tavily search tool from LangChain's community toolset

# Create a search tool instance
search = TavilySearchResults(max_results=2)
# This limits the number of search results returned by Tavily (in this case, 2)

# Test it out directly (outside of the agent)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# Should print real-time search results about the weather in San Francisco

# Add the search tool to a list — this is what we'll give to the agent later
tools = [search]

[{'title': 'Wednesday, April 16, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-16', 'content': 'San Francisco, California Weather: Wednesday, April 16, 2025. Cloudy weather, overcast skies with clouds. Day 66. Night 54.', 'score': 0.933969}, {'title': 'Weather in San Francisco in April 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/april-2025/', 'content': "Weather in San Francisco in April 2025\n\nSan Francisco Weather Forecast for April 2025 is based on long term prognosis and previous years' statistical data.\n\nApril\n\n+50°\n\n+46°\n\n+46°\n\n+46°\n\n+50°\n\n+50°\n\n+54°\n\n+54°\n\n+52°\n\n+52°\n\n+55°\n\n+50°\n\n+48°\n\n+52°\n\n+52°\n\n+54°\n\n+54°\n\n+52°\n\n+50°\n\n+52°\n\n+50°\n\n+50°\n\n+52°\n\n+52°\n\n+52°\n\n+52°\n\n+55°\n\n+52°\n\n+52°\n\n+54°\n\nExtended weather forecast in San Francisco\n\nWeather in large and nearby cities\n\nWeather in Washington, D.C.+59°

In [13]:
%pip install -U langchain langchain-core langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [21]:
# ========================
# ✅ Step 1: Set API Key
# ========================

import os          # Allows setting environment variables
import getpass     # Securely collects sensitive input like API keys

# Prompt the user to enter their OpenAI API key (input will be hidden)
# This key is required to use OpenAI's language models through LangChain
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# ========================
# ✅ Step 2: Load Language Model
# ========================

from langchain_openai import ChatOpenAI  # Import LangChain’s wrapper for OpenAI chat models

# Initialize the language model
# "gpt-4o-mini" is a smaller, faster GPT-4 model (good for prototyping and saving cost)
model = ChatOpenAI(model="gpt-4o-mini")

# ========================
# ✅ Ready to Use!
# ========================
# Now you can use `model.invoke(...)` to:
# - Have conversations
# - Run logic through agents or workflows
# - Stream responses
# - Or power tool-using agents in LangGraph

Enter your OpenAI API key:  ········


In [23]:
from langchain_core.messages import HumanMessage
# HumanMessage is a standardized message format used by LangChain to represent user input

# Send a simple greeting to the model
response = model.invoke([HumanMessage(content="hi!")])
# .invoke() runs the model on the given input and returns a response object

# Display just the content of the response (not metadata or other fields)
response.content

'Hello! How can I assist you today?'

In [25]:
# Bind the available tools to the model using LangChain's `.bind_tools()` method.
# This allows the LLM to:
# - See the names/descriptions of available tools
# - Choose which tool to use (if any) during reasoning
# - Automatically call the tool with the right inputs when needed
model_with_tools = model.bind_tools(tools)

In [27]:
from langchain_core.messages import HumanMessage
# Import the HumanMessage class to format the input in a structure the model expects.

# Call the model (now bound with tools) using a basic greeting.
# This helps test:
# - If the model responds normally
# - Whether it tries to call any tools (it shouldn't for just "Hi!")
response = model_with_tools.invoke([
    HumanMessage(content="Hi!")
])

# Display the model's text response.
print(f"ContentString: {response.content}")     # The assistant's reply to "Hi!"

# Check if the model attempted to call a tool (it probably won't in this case).
print(f"ToolCalls: {response.tool_calls}")       # If the model wanted to use a tool, you'd see it listed here

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [29]:
#Import the prebuilt ReAct agent constructor from LangGraph
from langgraph.prebuilt import create_react_agent

#Create the agent executor by combining:
# - Your base LLM (`model`, NOT `model_with_tools`)
# - Your list of tools (in this case, just TavilySearchResults)
# `create_react_agent` automatically binds tools and handles reasoning logic
agent_executor = create_react_agent(model, tools)

#Now let's test the agent with a simple input that does NOT require tool use
#This allows us to verify that the base reasoning works without triggering tool calls
response = agent_executor.invoke({
    "messages": [HumanMessage(content="hi!")]
})

#`response["messages"]` will return the full message history including the AI's reply
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='8175d47b-757f-4f24-b6cc-e718f4d26d37'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BN3qWvo4Psxcabwm7oXUp5yyfjoz7', 'finish_reason': 'stop', 'logprobs': None}, id='run-59b856d5-51bd-4727-b8b1-dc6dafbc7856-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [33]:
# Test the agent with a query that SHOULD trigger tool use (Tavily search)
# This message asks for external information — weather in Minneapolis — which the LLM doesn’t know natively.

response = agent_executor.invoke({
    "messages": [HumanMessage(content="whats the weather in Minneapolis?")]
})

# This returns the full message history, including:
# - the user's message
# - the agent's thoughts/actions
# - the final answer (after tool use)
response["messages"]

[HumanMessage(content='whats the weather in Minneapolis?', additional_kwargs={}, response_metadata={}, id='86b119c7-685e-4e48-813c-b55f859491ea'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GG7MOTsVWZB12E0exNRpX9Ck', 'function': {'arguments': '{"query":"current weather in Minneapolis"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BN3uOF6guYbuJPfhemS0KGWA5EUjp', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ff231abc-c6bd-499b-aac5-dd9e60b8b87f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'c

In [35]:
from langchain_core.messages import HumanMessage

# Stream the agent's reasoning and tool use in real time
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Minneapolis?")]}
):
    print(chunk)  # Print each intermediate message (Thought, Action, Observation, Final Answer)
    print("----")  # Separator for clarity

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RXmXkHBLX1eJAeL8SopO1382', 'function': {'arguments': '{"query":"current weather in Minneapolis"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BN3wIbqp8alh0CU0Fc91tWRgEi0wt', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1911f39f-9e18-4b19-be8e-69ff3acb0191-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Minneapolis'}, 'id': 'call_RXmXkHBLX1eJAeL8SopO1382', 'type': 'tool_call'}], usage_metadata={'input_tokens

In [39]:
import asyncio
from langchain_core.messages import HumanMessage

# This function will run the event stream asynchronously
async def stream_agent_events():
    # Start streaming events from the agent
    async for event in agent_executor.astream_events(
        {"messages": [HumanMessage(content="whats the weather in Minneapolis?")]},
        version="v1",  # Version of the stream format
    ):
        kind = event["event"]  # Event type (e.g. on_tool_start, on_chain_end)

        if kind == "on_chain_start":
            if event["name"] == "Agent":
                print(f"\n Starting agent: {event['name']}")
                print(f"   Input: {event['data'].get('input')}\n")

        elif kind == "on_chat_model_stream":
            # Print each token as it is generated (like real-time typing)
            content = event["data"]["chunk"].content
            if content:
                print(content, end="|")  # Add a pipe to simulate streaming

        elif kind == "on_tool_start":
            print("\n Starting tool:")
            print(f"   Name: {event['name']}")
            print(f"   Input: {event['data'].get('input')}\n")

        elif kind == "on_tool_end":
            print(f" Done with tool: {event['name']}")
            print(f"   Output: {event['data'].get('output')}\n")

        elif kind == "on_chain_end":
            if event["name"] == "Agent":
                print("\n Agent finished.")
                print(f"   Output: {event['data'].get('output')['output']}")
                print("--")

# Call the function if you're in a notebook or script
await stream_agent_events()


 Starting tool:
   Name: tavily_search_results_json
   Input: {'query': 'current weather in Minneapolis'}

 Done with tool: tavily_search_results_json
   Output: content='[{"title": "Weather in Minneapolis", "url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Minneapolis\', \'region\': \'Minnesota\', \'country\': \'United States of America\', \'lat\': 44.98, \'lon\': -93.2636, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1744836265, \'localtime\': \'2025-04-16 15:44\'}, \'current\': {\'last_updated_epoch\': 1744835400, \'last_updated\': \'2025-04-16 15:30\', \'temp_c\': 14.4, \'temp_f\': 57.9, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 11.2, \'wind_kph\': 18.0, \'wind_degree\': 141, \'wind_dir\': \'SE\', \'pressure_mb\': 1014.0, \'pressure_in\': 29.93, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 30, \'cloud\': 75, \'feelslike_c\': 13

In [43]:
# Step 1: Import and initialize in-memory checkpointer
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# This saves the chat history in memory (RAM).
# Great for testing multi-turn conversations or user-specific threads.

# Step 2: Re-create the agent, this time with memory enabled
agent_executor = create_react_agent(
    model,       # Your language model (e.g., GPT-4o-mini)
    tools,       # List of tools the agent can use (e.g., Tavily search)
    checkpointer=memory  # Enables memory with in-RAM persistence
)

# Step 3: Define a conversation thread ID
config = {"configurable": {"thread_id": "abc123"}}
# This keeps your conversation scoped — memory is tied to this ID.

# Step 4: Start a new conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Dan!")]},  # First user message
    config  # Scoped memory config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Dan! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 83, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BN43UEPd1FOJGjnjHsfyJfgpopPP7', 'finish_reason': 'stop', 'logprobs': None}, id='run-84a82c58-002c-451e-82dc-c0d6134f4d24-0', usage_metadata={'input_tokens': 83, 'output_tokens': 12, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [47]:
# Stream the agent's response to a follow-up message
# This message asks the assistant to recall the user's name
# Because we're using the same `thread_id` in `config`, memory is preserved

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]},  # The follow-up prompt
    config,  # Same thread_id as before, so memory of "hi im Dan!" is accessible
):
    # Streamed output will include intermediate reasoning steps and final answer
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is still Dan! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 124, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BN45gMk6XtHw14QO5fgePIJIp44H5', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff75e1b8-0f4f-422e-ac1e-5279f4b5dbb4-0', usage_metadata={'input_tokens': 124, 'output_tokens': 15, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [49]:
#  Start a brand new conversation by changing the thread_id
# This simulates a fresh session where the assistant does not remember previous messages

config = {"configurable": {"thread_id": "xyz123"}}  # New conversation thread

# Ask the same question as before, but without prior context
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]},  # No name has been given in this thread
    config,
):
    print(chunk)  # Should respond with something like "I'm not sure" or "You haven't told me your name yet"
    print("----")

{'agent': {'messages': [AIMessage(content='I don’t have access to personal information unless you provide it to me. Can you please tell me your name?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 84, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BN45rI4gzIcNUVKSoO47zOpggVEWI', 'finish_reason': 'stop', 'logprobs': None}, id='run-15626dce-bff6-4f2b-b393-6503ce29da0e-0', usage_metadata={'input_tokens': 84, 'output_tokens': 25, 'total_tokens': 109, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
